In [29]:
import os
import requests
from zipfile import ZipFile

def download():
    os.makedirs("data")    

    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip"
    response = requests.get(url)#указанного ниже URL вызывается метод get().
    #Метод GET указывает на то, что происходит попытка извлечь данные из определенного ресурса

    with open("data/household_power_consumption.zip", 'wb') as file: 
        for line in response:
            file.write(line)

    with ZipFile("data/household_power_consumption.zip") as myzip: #zipFile
        myzip.extractall("data")#извлечь все файлы из архива ..z.extract('file') # Извлечь файл из архива



In [28]:
download()

In [337]:
import numpy as np
import pandas as pd
import timeit

## Dataframe

In [372]:
%%time
table = pd.read_csv("data/household_power_consumption.txt", sep=';', engine='python')
table.columns = ['Date', 'Time', 'active', 'reactive', 'voltage', 'intensity', 'metering1', 'metering2', 'metering3']


Wall time: 32.4 s


In [373]:
%%time
table.drop(table[table.isna().any(axis=1)].index, inplace=True)

Wall time: 1.79 s


In [374]:
%%time
for column in table.columns[slice(2,9)]:
    table[column] = table[column].astype('float32')
    
table.dtypes

Wall time: 4.92 s


Date          object
Time          object
active       float32
reactive     float32
voltage      float32
intensity    float32
metering1    float32
metering2    float32
metering3    float32
dtype: object

## (1)
 Обрати всі рядки, у яких загальна актив. споживана потужн. > 5 кВт

In [375]:
%%time
table[table['active'] > 5]

Wall time: 65.8 ms


,Date,Time,active,reactive,voltage,intensity,metering1,metering2,metering3
1,16/12/2006,17:25:00,5.360,0.436,233.630005,23.000000,0.0,1.0,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.289993,23.000000,0.0,2.0,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740005,23.000000,0.0,1.0,17.0
11,16/12/2006,17:35:00,5.412,0.470,232.779999,23.200001,0.0,1.0,17.0
12,16/12/2006,17:36:00,5.224,0.478,232.990005,22.400000,0.0,1.0,16.0
...,...,...,...,...,...,...,...,...,...
2069356,22/11/2010,18:40:00,5.408,0.150,231.500000,23.600000,48.0,0.0,0.0
2069357,22/11/2010,18:41:00,5.528,0.144,232.479996,24.600000,53.0,0.0,0.0
2071586,24/11/2010,07:50:00,5.172,0.050,235.179993,22.000000,0.0,38.0,17.0
2071587,24/11/2010,07:51:00,5.750,0.000,234.399994,24.600000,0.0,39.0,17.0


## (2)
Обрати всі рядки, у яких вольтаж > 235 В.

In [376]:
%%time
table[table['voltage'] >235]

Wall time: 198 ms


,Date,Time,active,reactive,voltage,intensity,metering1,metering2,metering3
4,16/12/2006,17:28:00,3.666,0.528,235.679993,15.8,0.0,1.0,17.0
5,16/12/2006,17:29:00,3.520,0.522,235.020004,15.0,0.0,2.0,17.0
6,16/12/2006,17:30:00,3.702,0.520,235.089996,15.8,0.0,1.0,17.0
7,16/12/2006,17:31:00,3.700,0.520,235.220001,15.8,0.0,1.0,17.0
14,16/12/2006,17:38:00,4.054,0.422,235.240005,17.6,0.0,1.0,17.0
...,...,...,...,...,...,...,...,...,...
2075254,26/11/2010,20:58:00,0.946,0.000,240.429993,4.0,0.0,0.0,0.0
2075255,26/11/2010,20:59:00,0.944,0.000,240.000000,4.0,0.0,0.0,0.0
2075256,26/11/2010,21:00:00,0.938,0.000,239.820007,3.8,0.0,0.0,0.0
2075257,26/11/2010,21:01:00,0.934,0.000,239.699997,3.8,0.0,0.0,0.0


## (3)
Обрати всі рядки, у яких 19<I<20 A, для них виявити ті, у яких пральна машина та холодильних
споживають більше, ніж бойлер та кондиціонер

In [377]:
%%time
temp = table[(table['intensity'] <= 20) & (table['intensity'] >= 19)]

Wall time: 49.9 ms


In [378]:
%%time
temp[temp['metering2'] > temp['metering3']]
temp

Wall time: 2.19 ms


,Date,Time,active,reactive,voltage,intensity,metering1,metering2,metering3
10,16/12/2006,17:34:00,4.448,0.498,232.860001,19.600000,0.0,1.0,17.0
24,16/12/2006,17:48:00,4.474,0.000,234.960007,19.400000,0.0,0.0,17.0
33,16/12/2006,17:57:00,4.512,0.000,233.619995,19.200001,0.0,0.0,17.0
45,16/12/2006,18:09:00,4.464,0.136,234.660004,19.000000,0.0,37.0,16.0
52,16/12/2006,18:16:00,4.524,0.076,234.199997,19.600000,0.0,9.0,17.0
...,...,...,...,...,...,...,...,...,...
2071589,24/11/2010,07:53:00,4.666,0.000,235.720001,19.799999,0.0,39.0,17.0
2071590,24/11/2010,07:54:00,4.694,0.000,236.779999,19.799999,0.0,39.0,18.0
2071591,24/11/2010,07:55:00,4.602,0.000,237.080002,19.400000,0.0,40.0,17.0
2071592,24/11/2010,07:56:00,4.536,0.000,237.029999,19.000000,0.0,39.0,17.0


## (4)
Обрати випадковим чином 500000 рядків (без повторів
елементів вибірки), для них обчислити середні величини усіх 3-х
груп споживання електричної енергії, а також

In [379]:
%%time
rand = table.sample(500000)
Mean1 = rand['metering1'].mean()
Mean2 = rand['metering2'].mean()
Mean3 = rand['metering3'].mean()

print(f"Mean1: {Mean1}",f"Mean2: {Mean2}", f"Mean3: {Mean3}")

Mean1: 1.1219979524612427 Mean2: 1.290611982345581 Mean3: 6.460398197174072
Wall time: 376 ms


## (5)
Обрати ті рядки, які після 18-00 споживають > 6
кВт за хвилину в середньому, серед відібраних визначити ті, у яких
основне споживання електроенергії у вказаний проміжок часу
припадає на пральну машину, сушарку, холодильник та освітлення
(група 2 є найбільшою), а потім обрати кожен третій результат із
першої половини та кожен четвертий результат із другої половини.

In [380]:
%%time
table['DateTime'] = pd.to_datetime(table['Date'] + '-' + table['Time'], format="%d/%m/%Y-%H:%M:%S")
table.drop(columns = ['Date', 'Time'], inplace=True)

Wall time: 15.2 s


In [381]:
table.set_index('DateTime', inplace=True)

In [382]:
table

,active,reactive,voltage,intensity,metering1,metering2,metering3
DateTime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.839996,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.630005,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.289993,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.740005,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.679993,15.8,0.0,1.0,17.0
...,...,...,...,...,...,...,...
2010-11-26 20:58:00,0.946,0.000,240.429993,4.0,0.0,0.0,0.0
2010-11-26 20:59:00,0.944,0.000,240.000000,4.0,0.0,0.0,0.0
2010-11-26 21:00:00,0.938,0.000,239.820007,3.8,0.0,0.0,0.0


In [383]:
%%time
time = table[(table.index.hour >= 18) & (table['active'] > 6)]
max_met2 = time[(time['metering2'] > time['metering1']) & (time['metering2'] >= time['metering3'])]


def temp(df : pd.DataFrame, n : int, start : int, end : int):
    cells = np.arange(start, end, n)      
    return df.iloc[ cells]

def fourth_2part(df : pd.DataFrame):
    start = np.ceil(len(df) / 2)
    end = len(df)
    return temp(df, 4, start, end)

def third_1part(df : pd.DataFrame):
    start = 0
    end = np.floor(len(df) / 2)
    return temp(df, 3, start, end)

Wall time: 346 ms


In [384]:
third_1part(max_met2)

,active,reactive,voltage,intensity,metering1,metering2,metering3
DateTime,,,,,,,
2006-12-16 18:05:00,6.052,0.192,232.929993,26.200001,0.0,37.0,17.0
2006-12-16 18:08:00,6.308,0.116,232.250000,27.000000,0.0,36.0,17.0
2006-12-28 20:58:00,6.386,0.374,236.630005,27.000000,1.0,36.0,17.0
2006-12-28 21:02:00,8.088,0.262,235.500000,34.400002,1.0,72.0,17.0
2006-12-28 21:05:00,7.230,0.152,235.220001,30.600000,1.0,73.0,17.0
...,...,...,...,...,...,...,...
2008-03-23 20:27:00,7.222,0.208,233.360001,30.799999,0.0,73.0,16.0
2008-03-23 20:30:00,6.952,0.178,234.240005,29.600000,0.0,72.0,17.0
2008-03-23 20:33:00,6.490,0.196,234.039993,27.600000,0.0,72.0,17.0


In [385]:
fourth_2part(max_met2)

,active,reactive,voltage,intensity,metering1,metering2,metering3
DateTime,,,,,,,
2008-04-20 21:03:00,7.798,0.328,231.759995,33.599998,1.0,71.0,17.0
2008-04-20 21:07:00,7.452,0.346,232.309998,32.200001,1.0,68.0,17.0
2008-04-20 21:11:00,6.436,0.382,232.100006,27.600000,1.0,37.0,17.0
2008-04-20 21:15:00,6.200,0.216,232.509995,26.600000,1.0,37.0,17.0
2008-05-18 20:30:00,6.760,0.236,234.899994,29.799999,2.0,52.0,28.0
...,...,...,...,...,...,...,...
2010-11-20 18:30:00,6.620,0.314,229.000000,29.200001,18.0,34.0,16.0
2010-11-20 18:34:00,6.266,0.360,229.820007,27.600000,14.0,35.0,16.0
2010-11-20 18:38:00,6.302,0.376,229.649994,27.799999,15.0,34.0,17.0
